# 트리의 앙상블

### 앙상블 학습이 무엇인지 이해하고 다양한 앙상블 학습 알고리즘을 실습을 통해 배웁니다.

## 정형 데이터와 비정형 데이터

우리가 다룬 생선의 데이터, 와인의 데이터는 scv파일에 가지런히 정리되어 있었다. 이런 형태의 데이터를 **정형 데이터**라고 부른다.

어떤 구조로 되어 있다는 뜻이다. 이런 데이터는 scv나 데이터베이스, 엑셀에 저장하기 쉽다.

이와 반대되는 데이터를 **비정형 데이터**라고 한다.

비정형 데이터는 데이터베이스나 엑셀로 표현하기 어려운 것들이다. 예를들어 텍스트 데이터, 사진, 디지털 음악 등이 있다.

정형 데이터를 다루는데 가장 뛰어난 성과를 내는 알고리즘이 **앙상블 학습**이다.

이 알고리즘은 대부분 결정 트리를 기반으로 만들어져 있다.

비정형 데이터에는 신경망 알고리즘을 사용한다.

## 랜덤 포레스트

**랜덤 포레스트**는 앙상블 학습의 대표주자로 안정적인 성능 덕분에 널리 사용되고 있다.

랜덤 포레스트는 결정 트리를 랜덤하게 만들어 결정트리의 **숲**을 만든다.

그리고 각 결정 트리의 예측을 사용해 최종 예측을 만든다.

**부트스트랩 샘플**은 데이터 세트에서 중복을 허용하여 데이터를 샘플링 하는 방식이다. 그래서 훈련 세트와 크기가 같다.

각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾는다.

분류 모델인 RandomForestClassifier는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택한다.

회귀 모델인 RandomForestRegressor는 전체 특성을 사용한다.

![사진](./forest.jpeg)

랜덤 포레스트는 기본적으로 100개의 결정 트리를 이런 방식으로 훈련한다.

분류일때는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼고

회귀일때는 단순히 각 트리의 예측을 평균한다.

랜덤 포레스트는 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 막아주고 검증세트와 테스트세트에서 안정적인 성능을 얻을 수 있다.

그럼 사이킷런의 RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용해 보자.

와인 데이터셋을 판다스로 불러오고 훈련 세트와 테스트 세트로 나눈다.

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(
data, target, test_size=0.2, random_state=42)

cross_validate() 함수를 사용해 교차 검증을 수행해보자.

return_train_score 매개변수를 True로 지정하면 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환된다.

훈련 세트와 검증 세트의 점수를 비교하면 과대적합을 파악하는데 용이하다.

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score'],), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


출력된 결과를 보면 훈련 세트에 다소 과대적합된 것 같다.

랜덤 포레스트는 결정 트리의 앙상블이기 때문에 결정 트리의 큰 장점중 하나인 특성중요도를 계산한다.

랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다.

랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력해보자.

In [7]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


앞의 1절 결정트리에서 만든 특성중요도와 비교해보자.

[0.12345626 0.86862934 0.0079144]

비교해 보면 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승했다.

이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정트리를 훈련하기 때문이다.

하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다.

이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다.

RandomForestClassifier에는 재미있는 기능이 하나 더 있는데 부트스트랩 샘플에 포함되지 않고 남는 샘플을 **OOB 샘플**이라고 한다.

이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. 마치 **검정 세트**의 역할을 하는것이다.

oob_score 매개변수를 True로 지정하면 랜덤 포레스트는 각 결정 트리의 OOB 점수를 평균하여 출력한다.

한번 출력해보자.

In [8]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


교차 검증에서 얻은 점수와 매우 비슷한 결과를 얻었다.

OOB점수를 사용하면 교차 검증을 대신할 수 있어서 결과적으로 훈련 세트에 더 많은 샘플을 사용할 수 있다.

## 엑스트라 트리

**엑스트라 트리**는 랜덤 포레스트와 매우 비슷하게 동작한다.

전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는데 사용한다.

차이점은 부트스트랩 샘플을 사용하지 않는다는 점이다.

결정 트리를 만들 때 전체 훈련 세트를 사용한다. 대신 노드를 분할할때 무작위로 분할한다.

하나의 결정 트리에서 특성을 무작위로 분할 한다면 성능은 낮아지겠지만 과대적합을 막고 검증 세트의 점수를 높이는 효과가 있다.

이 모델의 교차 검증 점수를 확인해 보자.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


랜덤 포레스트와 비슷한 결과를 얻었다. 이 예제는 특성이 많지 않아 두 모델의 차이가 크지 않다.

엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트 보다 더 많은 결정트리를 훈련해야 한다.

하지만 빠른 계산속도가 엑스트라 트리의 장점이다.

엑스트라 트리도 특성 중요도를 제공한다.

결과를 보면 엑스트라 트리도 결정트리보다 당도에 대한 의존성이 작다.

In [10]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

**그레이디언트 부스팅**은 깊이가 얕은 결정 트리를 사용하여 이전트리의 오차를 보완한다.

기본적으로 깊이가 3인 결정트리를 100개 사용한다.

그래서 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있다.

그레이디언트 부스팅은 결정 트리를 계속 추가하면서 가장 낮은 곳으로 찾아 이동한다.

손실함수의 낮은 곳으로 천천히 조금씩 이동해야 한다고 했는데 그래서 깊이가 얕은 트리를 사용한다.

또 학습률 매개변수로 속도를 조절한다.

GradientBoostingClassifier를 사용해 와인 데이터셋의 교차 검증 점수를 확인해보자.

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


그레이디언트 부스팅은 결정 트리의 개수를 늘려도 과대적합에 매우 강하다.

학습률을 증가시키고 트리의 개수를 늘리면 조금 더 성능이 향상될 수 있다.

In [13]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2,
                               random_state=42)
scores = cross_validate(gb, train_input, train_target,
                       return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


결정 트리 개수를 500개로 늘렸지만 과대적합을 잘 억제하고 있다.

그레이디언트 부스팅도 특성 중요도를 제공한다.

그레이디언트 부스팅이 랜덤 포레스트 보다 일부 특성(당도)에 더 집중한다.

In [14]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


재미있는 매개변수가 하나 있는데 트리 훈련에 사용할 훈련 세트의 비율을 정하는 subsample이다.

이 매개변수의 기본값은 1로 전체 훈련 세트를 사용한다. 1보다 작으면 훈련세트의 일부를 사용한다. 마치 확률적 경사 하강법이나 미니배치 경사하강법과 비슷하다.

그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 얻을 수있지만 순서대로 트리를 추가하기 때문에 훈련 속도가 느리다.

## 히스토그램 기반 그레이디언트 부스팅

**히스토그램 기반 그레이디언트 부스팅**은 정형 데이터를 다루는 머신러닝 알고리즘이다.

히스토그램 기반 그레이디언트 부스팅은 입력 특성을 256개의 구간으로 나눈다. 따라서 최적의 분할을 매우 빠르게 찾을 수 있다.

256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용한다. 그래서 따로 전처리할 필요가 없다.

기본 매개변수에서 안정적인 성능을 얻을수 있고 부스팅 반복 횟수를 지정하는 max_iter를 사용한다.

그럼 와인 데이터셋에 HistGradientBoostingClassifier 클래스를 적용해 보자.

In [18]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                       return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


과대적합을 잘 억제 하면서 그레이디언트 부스팅보다 조금 더 높은 성능을 제공한다.

특성 중요도를 계산하기위해 permutation_importance() 함수를 사용한다. 

이 함수는 특성을 하나씩 랜덤하게 섞어서 모델의 성능이 변화하는지를 관찰하여 어떤 특성이 중요한지를 계산한다.

n_repeats 매개변수는 랜덤하게 섞을 횟수를 지정한다.

In [19]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target,
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


permutation_importance() 함수가 반환하는 객체는 특성중요도, 평균, 표준편차를 담고 있다.

평균을 출력해보면 랜덤 포레스트와 비슷한 비율임을 알 수 있다.

이번에는 테스트 세트에서 특성 중요도를 계산해보자.

In [20]:
result = permutation_importance(hgb, test_input, test_target, 
                               n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


테스트 세트의 결과를 보면 그레이디언트 부스팅과 비슷하게 조금 더 당도에 집중하고 있다는 것을 알 수 있다.

이런 분석을 통해 모델을 실전에 투입했을 때 어떤 특성에 관심을 둘지 예상할 수 있다.

그럼 HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종적으로 확인해보자.

In [21]:
hgb.score(test_input, test_target)

0.8723076923076923

테스트 세트에서는 약 87%의 정확도를 얻었다.

앙상블 모델은 확실히 단일 결정 트리보다 좋은 결과를 얻을 수 있다.

## 앙상블 학습을 통한 성능 향상

**앙상블 학습**은 강력하고 뛰어난 성능을 제공한다.

앙상블 학습 알고리즘 중 랜덤포레스트, 엑스트라 트리, 그레이디언트 부스팅, 히스토그램 기반 그레이디언트 부스팅을 다루었다.

**랜덤 포레스트**는 가장 대표적인 앙상블 학습 알고리즘이다. 성능이 좋고 안정적이다.

결정트리를 훈련하기 위해 부트스트랩 샘플을 만들고 전체 특성 중 일부를 랜덤하게 선택하여 결정 트리를 만든다.

**엑스트라 트리**는 부트스트랩을 사용하지 않고 노드를 분할할때 랜덤하게 분할한다. 훈련 속도가 빠르지만 보통 더 많은 트리가 필요하다.

**그레이디언트 부스팅**은 깊이가 얕은 트리를 연속적으로 추가하여 손실 함수를 최소화하는 앙상블 방법이다.

훈련속도가 조금 느리다, 학습률 매개변수를 조정하여 모델의 복잡도를 제어할 수 있다.

학습률 매개변수가 크면 복잡하고 훈련 세트에 과대적합된 모델을 얻을 수 있다.

**히스토그램 기반 그레이디언트 부스팅 알고리즘**은 훈련 데이터를 256개의 구간으로 변환하여 노드 분할 속도가 매우빠르다.

지금 까지는 입력과 타깃이 준비된 문제를 풀었다. 이런 머신러닝 분야를 **지도 학습**이라고 한다.

타깃이 없으면 어떨까?? 다음장에서 이에 대해 배워보자